<h1><center>HW2 Scrape Hotel Reviews</center></h1>

Choose one hotel at tripadvisor.com (e.g.https://www.tripadvisor.com/Hotel_Review-g60763-d23448880-Reviews-Motto_by_Hilton_New_York_City_Chelsea-New_York_City_New_York.html)

- Q1. Write a function to scrape all **reviews on the first page**, including, 
    - **username** (see (1) in Figure)
    - **review date** (see (2) in Figure)
    - **rating** (see (3) in Figure)
    - **title** (see (4) in Figure)
    - **review text** (see (5) in Figure. For each review text, need to get the **complete text**.)
    - **date of stay**(see (6) in Figure)
    - If a field, e.g. rating, is missing, use `None` to indicate it. 
- `Function Input`: hotel page URL
- `Function Output`: save all reviews as a DataFrame of columns. E.g., for the given URL, you can get 10 reviews.

    
![alt text](tripadvisor.png "TripAdvisor")

In [1]:
import requests
from bs4 import BeautifulSoup  
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re


In [ ]:
driver = webdriver.Safari()

In [216]:
def getReviews(page_url):
    reviews_tb = pd.DataFrame()
    
    driver = webdriver.Safari()
    driver.get(page_url)

    driver.implicitly_wait(3)

    reviews_tab = driver.find_element(By.XPATH, '//*[@id="component_18"]/div/div[3]/div[3]')
    reviews_tab.click()
    time.sleep(5)
    
   

    return reviews_tb


In [217]:
page_url = 'https://www.tripadvisor.com/Hotel_Review-g60763-d23448880-Reviews-Motto_by_Hilton_New_York_City_Chelsea-New_York_City_New_York.html'
getReviews(page_url)


,username,review_date,rating,title,review_text,date_of_stay
0,Vic S,Today,5.0,First Choice Every Time,We have stayed at this hotel multiple times in...,February 2023
1,Alice L,Feb 2022,5.0,Perfect for Us,We recently chose Motto for an overnight in NY...,February 2022
2,Kari C,Today,5.0,Fabulous Hotel,We recently stayed at the Motto by Hilton in C...,January 2023
3,SamBBer,Today,5.0,Beautiful place & perfect location,We like to enjoy a weekend in NYC from time to...,December 2022
4,joel l,Yesterday,3.0,"No closet, no desk, no dresser",I understand that hotel rooms are often on the...,February 2023
5,lauryn r,Yesterday,5.0,Yasmin is amazing!,Yasmin was absolutely amazing!!! She really he...,January 2023
6,Tom I,Feb 2,5.0,Amazing location and view.,"Motto Chelsea was just amazing, the help of Ze...",January 2023
7,Passport58331943467,Feb 2,5.0,Amazing property and staff!,I booked a stay to celebrate my partner’s birt...,January 2023
8,Wade4020,Feb 2,5.0,Just a great comfortable experience.,The last few times I have traveled to NYC I ha...,February 2023
9,Paradise99206,Feb 1,4.0,Awesome location,A great location. Rooms were clean and functio...,January 2023


In [218]:
driver.quit()

### Bonus point. 
* Modify the function you defined in Q1 to Scrape **all reviews on the first five page**.


In [243]:
def getReviews2(page_url):
    
    # add your code
    
    
    return reviews_tb

    

In [244]:
page_url = 'https://www.tripadvisor.com/Hotel_Review-g60763-d23448880-Reviews-Motto_by_Hilton_New_York_City_Chelsea-New_York_City_New_York.html'
getReviews2(page_url)

,username,review_date,rating,title,review_text,date_of_stay
0,Vic S,Today,5.0,First Choice Every Time,We have stayed at this hotel multiple times in...,February 2023
1,Alice L,Feb 2022,5.0,Perfect for Us,We recently chose Motto for an overnight in NY...,February 2022
2,Kari C,Today,5.0,Fabulous Hotel,We recently stayed at the Motto by Hilton in C...,January 2023
3,SamBBer,Today,5.0,Beautiful place & perfect location,We like to enjoy a weekend in NYC from time to...,December 2022
4,joel l,Yesterday,3.0,"No closet, no desk, no dresser",I understand that hotel rooms are often on the...,February 2023
5,lauryn r,Yesterday,5.0,Yasmin is amazing!,Yasmin was absolutely amazing!!! She really he...,January 2023
6,Tom I,Feb 2,5.0,Amazing location and view.,"Motto Chelsea was just amazing, the help of Ze...",January 2023
7,Passport58331943467,Feb 2,5.0,Amazing property and staff!,I booked a stay to celebrate my partner’s birt...,January 2023
8,Wade4020,Feb 2,5.0,Just a great comfortable experience.,The last few times I have traveled to NYC I ha...,February 2023
9,Paradise99206,Feb 1,4.0,Awesome location,A great location. Rooms were clean and functio...,January 2023


In [245]:
driver.quit()